In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

Construimos el dataset combinado de pares (D,A) para 2025-Q1 a 2025-Q3 y guardamos en .csv

Deduplicaremos dentro de cada reporte (por safetyreportid) para que cada reporte cuente como una sola evidencia de que un fármaco $D$ y una reacción $A$ aparecen juntos, evitando inflar artificialmente los conteos.

En FAERS es común que, dentro del mismo reporte, un mismo fármaco o reacción aparezca repetido (por múltiples entradas, dosis, formulaciones, roles del medicamento, o registros duplicados).

In [2]:
# Cargamos los 3 trimestres
BASE_ROOT = Path("data_processed")
quarters = ["q1_2025_sample100k", "q2_2025_sample100k", "q3_2025_sample100k"]

all_pairs = []

for q in quarters:
    base = BASE_ROOT / q

    reports = pd.read_csv(base/f"{q.split('_')[0]}_reports_100k.csv", dtype={"safetyreportid":"string"})
    drugs = pd.read_csv(base/f"{q.split('_')[0]}_drugs_100k.csv", dtype={"safetyreportid":"string"})
    reactions = pd.read_csv(base/f"{q.split('_')[0]}_reactions_100k.csv", dtype={"safetyreportid":"string"})

    # Limpieza 
    for col in ["medicinalproduct", "activesubstancename"]:
        drugs[col] = drugs[col].astype("string").str.strip().str.upper()
    reactions["reaction_pt"] = reactions["reaction_pt"].astype("string").str.strip().str.upper()

    # D principal: sustancia activa 
    drugs["drug_key"] = drugs["activesubstancename"].fillna(drugs["medicinalproduct"]).astype("string")
    drugs["drug_key"] = drugs["drug_key"].str.strip().str.upper()

    # Deduplicar dentro de cada reporte
    drugs_u = drugs[["safetyreportid","drug_key"]].dropna().drop_duplicates()
    reac_u  = reactions[["safetyreportid","reaction_pt"]].dropna().drop_duplicates()

    # Pares por trimestre
    pairs = drugs_u.merge(reac_u, on="safetyreportid", how="inner")
    pairs["quarter"] = q.split("_")[0].upper()  # Q1/Q2/Q3

    # Guardar pares del trimestre
    pairs.to_csv(base/f"{q.split('_')[0]}_pairs_100k.csv", index=False)

    all_pairs.append(pairs)

# Unimos los 3 trimestres
pairs_2025_q1q3 = pd.concat(all_pairs, ignore_index=True)
out_path = BASE_ROOT / "pairs_2025_q1q3_300k.csv"
pairs_2025_q1q3.to_csv(out_path, index=False)

print("Saved combined pairs:", out_path)
print("rows:", len(pairs_2025_q1q3))
print("unique drugs:", pairs_2025_q1q3["drug_key"].nunique())
print("unique reactions:", pairs_2025_q1q3["reaction_pt"].nunique())
print("quarters counts:\n", pairs_2025_q1q3["quarter"].value_counts())


Saved combined pairs: data_processed/pairs_2025_q1q3_300k.csv
rows: 8081574
unique drugs: 20153
unique reactions: 12187
quarters counts:
 quarter
Q3    2878325
Q1    2837090
Q2    2366159
Name: count, dtype: int64


In [3]:
pairs_2025_q1q3

,safetyreportid,drug_key,reaction_pt,quarter
0,24795582,BUDESONIDE\FORMOTEROL FUMARATE DIHYDRATE,ASTHMA,Q1
1,24795583,DUPILUMAB,NASAL POLYPS,Q1
2,24795583,DUPILUMAB,CONDITION AGGRAVATED,Q1
3,24795583,DUPILUMAB,INAPPROPRIATE SCHEDULE OF PRODUCT ADMINISTRATION,Q1
4,24795584,ROSUVASTATIN CALCIUM,DRUG INTERACTION,Q1
...,...,...,...,...
8081569,25869809,MYCOPHENOLATE MOFETIL,LUNG TRANSPLANT,Q3
8081570,25869833,PEGFILGRASTIM-APGF,COVID-19,Q3
8081571,25869833,PEGFILGRASTIM-APGF,NEUTROPENIA,Q3
8081572,25869834,MACITENTAN,LUNG TRANSPLANT,Q3


El archivo creado contiene 8,081,574 filas, donde cada fila representa la coocurrencia (drug_key, reaction_pt) dentro de un mismo safetyreportid (tras deduplicar por reporte). En total se observan 20,153 fármacos distintos (definidos principalmente por sustancia activa) y 12,187 reacciones distintas (Preferred Terms). La contribución por trimestre en número de pares es: Q3 = 2,878,325, Q1 = 2,837,090 y Q2 = 2,366,159, lo cual refleja diferencias de volumen/combinaciones reportadas entre trimestres y justifica analizar estabilidad temporal de las señales.

Calculamos las señales de desproporcionalidad para cada par (D,A): la tabla 2×2 $(n_{11},n_{10},n_{01},n_{00})$ y a partir de ahí ROR (con intervalo de confianza). También, añadimos un filtro mínimo de soporte ($n_{11}\geq 5$) se crea para evitar señales inestables causadas por conteos muy pequeños.

Aplicamos la corrección de **Haldane–Anscombe** de forma uniforme a todos los pares para evitar estimaciones indefinidas o infinitas del **ROR** cuando alguna celda es cero (situación frecuente en datos de farmacovigilancia por la cola larga y la rareza de muchos eventos). Usarla en todos los casos mantiene una fórmula **consistente y estable numéricamente** (sin cambiar de regla solo para algunos pares), reduce la sensibilidad a conteos muy pequeños y permite calcular de manera robusta el intervalo de confianza del ROR en todo el conjunto analizado; además, cuando los conteos son moderados o grandes, el efecto de la corrección es prácticamente despreciable.

Finalmente, se hace un ranking para priorizar qué pares (fármaco–reacción) revisar primero y convertir una lista enorme de señales en un conjunto manejable e interpretable. En FAERS hay decenas o cientos de miles de pares evaluables; aun con filtros, quedan muchísimas asociaciones positivas. Ordenarlas por un criterio estadístico (por ejemplo, ROR y especialmente su límite inferior del IC95%, ROR_L95) permite:

- Enfocar el análisis en las asociaciones con evidencia más fuerte y más robusta (no solo valores altos por azar).

- Reducir falsos positivos: un ROR alto con IC amplio puede ser inestable; el ranking por ROR_L95 favorece señales con soporte y precisión.

- Construir Top-N (Top-200) para inspección clínica, validación del pipeline y comparación de estabilidad por trimestre.


In [8]:
df = pairs_2025_q1q3.copy()

#  Universo GLOBAL de informes
N = df["safetyreportid"].nunique()

# deduplicar a nivel de informe
df4 = df.drop_duplicates(["safetyreportid","drug_key","reaction_pt"])
dfD = df.drop_duplicates(["safetyreportid","drug_key"])
dfA = df.drop_duplicates(["safetyreportid","reaction_pt"])


# Conteos básicos:
# n11: # reportes donde aparece (D,A)
n11 = (df4.groupby(["drug_key","reaction_pt"])
          .size()
          .rename("n11")
          .reset_index())

# n1.: # reportes donde aparece D (cualquier A)
n1dot = (dfD.groupby("drug_key")
           .size()
           .rename("n1dot")
           .reset_index())

# n.1: # reportes donde aparece A (cualquier D)
ndot1 = (dfA.groupby("reaction_pt")
           .size()
           .rename("ndot1")
           .reset_index())


# Creamos tabla 2x2 por par

sig = (n11.merge(n1dot, on="drug_key", how="left")
          .merge(ndot1, on="reaction_pt", how="left"))

sig["N"] = int(N)
sig["n10"] = sig["n1dot"] - sig["n11"] # D sin A
sig["n01"] = sig["ndot1"] - sig["n11"] # A sin D
sig["n00"] = sig["N"] - sig["n11"] - sig["n10"] - sig["n01"] # ni D ni A


#  filtro mínimo de soporte
MIN_N11 = 5
sig = sig[sig["n11"] >= MIN_N11].copy()

# ROR + IC95% (con corrección 0.5 tipo Haldane-Anscombe para evitar division by zero)
a = sig["n11"].astype(float) + 0.5
b = sig["n10"].astype(float) + 0.5
c = sig["n01"].astype(float) + 0.5
d = sig["n00"].astype(float) + 0.5

sig["ROR"] = (a*d)/(b*c)
sig["SE_logROR"] = np.sqrt(1/a + 1/b + 1/c + 1/d)
sig["ROR_L95"] = np.exp(np.log(sig["ROR"]) - 1.96*sig["SE_logROR"])
sig["ROR_U95"] = np.exp(np.log(sig["ROR"]) + 1.96*sig["SE_logROR"])

# PRR (solo para comparar)
sig["PRR"] = (sig["n11"] / (sig["n11"] + sig["n10"])) / (sig["n01"] / (sig["n01"] + sig["n00"]))

# Ranking para prioriza los pares con ROR_L95 > 1 y luego ROR
sig["is_signal"] = sig["ROR_L95"] > 1
sig = sig.sort_values(["is_signal","ROR"], ascending=[False, False])

print("N reports:", N)
print("signals rows:", len(sig))
sig.head(10)

N reports: 293664
signals rows: 236810


,drug_key,reaction_pt,n11,n1dot,ndot1,N,n10,n01,n00,ROR,SE_logROR,ROR_L95,ROR_U95,PRR,is_signal
419466,DICYCLOMINE HYDROCHLORIDE\PHENOBARBITAL,THYROID ATROPHY,6,6,8,293664,0,2,293656,1.527014e+06,1.598077,66608.936677,3.500688e+07,1.468290e+05,True
764037,LEVOMEFOLIC ACID\METHYLCOBALAMIN\TURMERIC,THYROID ATROPHY,6,6,8,293664,0,2,293656,1.527014e+06,1.598077,66608.936677,3.500688e+07,1.468290e+05,True
1058131,PREDNAZOLINE,BRAIN SCAN ABNORMAL,6,6,8,293664,0,2,293656,1.527014e+06,1.598077,66608.936677,3.500688e+07,1.468290e+05,True
1058132,PREDNAZOLINE,CEREBRAL ARTERY THROMBOSIS,6,6,8,293664,0,2,293656,1.527014e+06,1.598077,66608.936677,3.500688e+07,1.468290e+05,True
347810,COPPER,FOREIGN BODY IN REPRODUCTIVE TRACT,467,745,467,293664,278,0,292919,9.834102e+05,1.416239,61264.325728,1.578562e+07,inf,True
763914,LEVOMEFOLATE GLUCOSAMINE,THYROID ATROPHY,5,5,8,293664,0,3,293656,9.229204e+05,1.570839,42465.761476,2.005809e+07,9.788633e+04,True
1145209,SALICYLAMIDE,CREATININE RENAL CLEARANCE INCREASED,8,8,14,293664,0,6,293650,7.680090e+05,1.507149,40036.453667,1.473252e+07,4.894267e+04,True
1216827,TECHNETIUM TC-99M SODIUM PERTECHNETATE,SCAN MYOCARDIAL PERFUSION ABNORMAL,8,10,9,293664,2,1,293653,6.656146e+05,1.088263,78863.195408,5.617865e+06,2.349232e+05,True
396753,DEVICE\GELATIN,MECHANICAL VENTILATION COMPLICATION,5,5,10,293664,0,5,293654,5.873090e+05,1.537413,28853.201176,1.195472e+07,5.873180e+04,True
1043204,POLYSORBATE 85,CREATININE RENAL CLEARANCE INCREASED,7,7,14,293664,0,7,293650,5.873010e+05,1.505546,30712.399020,1.123072e+07,4.195100e+04,True


Para reducir y presentar un resultado global, nos quedamos con un ranking interpretable de señales y guardamos un archivo “final” para trabajar (Top-N + filtros estándar).

Para esto haremos lo siguiente:

- filtrar señales “positivas” (IC95% inferior > 1),

- aplicar un umbral más estricto de soporte (n11),

- sacar Top-200 (N=200),

- guardar CSVs.

In [9]:
out_dir = Path("data_processed")
out_dir.mkdir(parents=True, exist_ok=True)

# Solo señales positivas (IC95% inferior > 1)
sig_pos = sig[sig["ROR_L95"] > 1].copy()

# Umbral de soporte más estricto 
MIN_N11_STRICT = 10
sig_pos = sig_pos[sig_pos["n11"] >= MIN_N11_STRICT].copy()

# Ranking final (por ROR o por ROR_L95)
sig_pos = sig_pos.sort_values(["ROR_L95","ROR","n11"], ascending=[False, False, False])

TOPN = 200
top = sig_pos.head(TOPN).copy()

print("Signals (ROR_L95>1):", len(sig_pos))
print(f"Top-{TOPN} saved")

# Guardamos
sig.to_csv(out_dir/"signals_global_q1q3_all.csv", index=False)
sig_pos.to_csv(out_dir/"signals_global_q1q3_positive.csv", index=False)
top.to_csv(out_dir/f"signals_global_q1q3_top{TOPN}.csv", index=False)

top.head(20)


Signals (ROR_L95>1): 112417
Top-200 saved


,drug_key,reaction_pt,n11,n1dot,ndot1,N,n10,n01,n00,ROR,SE_logROR,ROR_L95,ROR_U95,PRR,is_signal
832406,MENTHOL,EXPOSURE TO CHEMICAL POLLUTION,135,182,137,293664,47,2,293480,334876.696842,0.654550,92836.861995,1.207951e+06,1.088463e+05,True
347810,COPPER,FOREIGN BODY IN REPRODUCTIVE TRACT,467,745,467,293664,278,0,292919,983410.170557,1.416239,61264.325728,1.578562e+07,inf,True
447241,DOLUTEGRAVIR\LAMIVUDINE\TENOFOVIR DISOPROXIL F...,CONGENITAL UMBILICAL HERNIA,14,22,16,293664,8,2,293640,200366.458824,0.765909,44655.037842,8.990412e+05,9.343155e+04,True
171520,BETA GLUCAN,TOTAL LUNG CAPACITY ABNORMAL,18,24,25,293664,6,7,293633,111430.148718,0.584155,35461.738350,3.501430e+05,3.146143e+04,True
348048,COPPER,REPRODUCTIVE COMPLICATION ASSOCIATED WITH DEVICE,364,745,364,293664,381,0,292919,559733.461337,1.416110,34878.991830,8.982529e+06,inf,True
326523,CLOFIBRATE,VAGINAL FLATULENCE,15,16,30,293664,1,15,293633,195755.666667,0.892021,34072.996247,1.124653e+06,1.835300e+04,True
55105,ALEFACEPT,NEUROLOGIC NEGLECT SYNDROME,17,20,29,293664,3,12,293632,117453.000000,0.650277,32834.991912,4.201374e+05,2.079978e+04,True
33185,ACLIDINIUM BROMIDE\FORMOTEROL FUMARATE,TOTAL LUNG CAPACITY INCREASED,15,29,17,293664,14,2,293633,125553.634483,0.730401,29998.513697,5.254832e+05,7.594009e+04,True
228050,CALCITRIOL\CALCIUM CARBONATE\ZINC,VAGINAL FLATULENCE,14,15,30,293664,1,16,293633,172027.707071,0.892324,29925.202807,9.889167e+05,1.712953e+04,True
1180195,SODIUM STEARATE,VAGINAL FLATULENCE,14,15,30,293664,1,16,293633,172027.707071,0.892324,29925.202807,9.889167e+05,1.712953e+04,True


Trabajaremos con el **Top-200** como un subconjunto manejable y reproducible de las señales globales porque el espacio total de asociaciones fármaco–evento es muy grande (en nuestro caso, tras el filtrado básico quedan más de **200 mil** pares evaluados y más de **200 mil** con señal positiva), y analizarlo completo no es práctico ni interpretativamente útil en una primera etapa. Al enfocarnos en las **200 señales mejor rankeadas** (ordenadas por magnitud y consistencia estadística mediante el estimador y el límite inferior del IC95% del ROR, junto con un umbral mínimo de soporte ($n_{11}$)), obtenemos un conjunto de alta prioridad donde las asociaciones son más estables, menos sensibles a fluctuaciones por conteos pequeños y más fáciles de inspeccionar y comunicar. Este enfoque permite validar el pipeline, revisar coherencia clínica, comparar estabilidad por trimestre y documentar hallazgos relevantes, dejando el resto de señales como un “universo” disponible para análisis posteriores o para profundizar en subgrupos específicos.


Diagnóstico rápido del Top-200 (qué tipo de reacciones dominan)

In [10]:
top = pd.read_csv(out_dir/"signals_global_q1q3_top200.csv", dtype={"drug_key":"string","reaction_pt":"string"})
top[["drug_key","reaction_pt","n11","ROR","ROR_L95","ROR_U95"]].head(20)

,drug_key,reaction_pt,n11,ROR,ROR_L95,ROR_U95
0,MENTHOL,EXPOSURE TO CHEMICAL POLLUTION,135,334876.696842,92836.861995,1.207951e+06
1,COPPER,FOREIGN BODY IN REPRODUCTIVE TRACT,467,983410.170557,61264.325728,1.578562e+07
2,DOLUTEGRAVIR\LAMIVUDINE\TENOFOVIR DISOPROXIL F...,CONGENITAL UMBILICAL HERNIA,14,200366.458824,44655.037842,8.990412e+05
3,BETA GLUCAN,TOTAL LUNG CAPACITY ABNORMAL,18,111430.148718,35461.738350,3.501430e+05
4,COPPER,REPRODUCTIVE COMPLICATION ASSOCIATED WITH DEVICE,364,559733.461337,34878.991830,8.982529e+06
5,CLOFIBRATE,VAGINAL FLATULENCE,15,195755.666667,34072.996247,1.124653e+06
6,ALEFACEPT,NEUROLOGIC NEGLECT SYNDROME,17,117453.000000,32834.991912,4.201374e+05
7,ACLIDINIUM BROMIDE\FORMOTEROL FUMARATE,TOTAL LUNG CAPACITY INCREASED,15,125553.634483,29998.513697,5.254832e+05
8,CALCITRIOL\CALCIUM CARBONATE\ZINC,VAGINAL FLATULENCE,14,172027.707071,29925.202807,9.889167e+05
9,SODIUM STEARATE,VAGINAL FLATULENCE,14,172027.707071,29925.202807,9.889167e+05


Creamos una versión “clínica” porque el Top-200 “full” suele mezclar dos tipos de términos de MedDRA:

1. **Eventos clínicos** (síntomas/diagnósticos: *pneumonia, rash, dyspnoea, etc.*)
2. **Términos administrativos o de uso/calidad** (p. ej. *OFF LABEL USE, DRUG INEFFECTIVE, PRODUCT DOSE OMISSION ISSUE, MEDICATION ERROR*).

Aunque ambos pueden ser relevantes, no significan lo mismo en farmacovigilancia. Los administrativos reflejan con frecuencia uso no aprobado, falta de efectividad, errores de medicación o problemas del producto, y tienden a aparecer muy arriba por patrones de reporte más que por un mecanismo biológico de evento adverso. Como nuestro objetivo principal es discutir señales clínicas (eventos adversos médicos), conviene separarlas para:

* **mejorar interpretabilidad** del Top-200 (que el ranking represente “eventos médicos”),
* **evitar que dominen** el listado términos de logística/uso,
* y poder presentar resultados en dos “capítulos”:
  **(a)** clínico (eventos adversos), **(b)** uso/efectividad/calidad (anexo o sección aparte).

Es importante aclarar que este filtro no cambia el cálculo de señales, solo crea una vista más adecuada para análisis clínico y comunicación.


In [12]:
admin_patterns = [
    "OFF LABEL", "DRUG INEFFECTIVE", "PRODUCT", "DEVICE", "MEDICATION ERROR",
    "MALFUNCTION", "QUALITY", "COUNTERFEIT", "ADMINISTRATION", "OMISSION",
    "UNAPPROVED", "THERAPEUTIC RESPONSE", "LACK OF", "INCORRECT"
]

mask_admin = top["reaction_pt"].str.contains("|".join(admin_patterns), case=False, na=False)
top_clin = top[~mask_admin].copy()

print("Top-200 total:", len(top))
print("Top-200 clínico (sin admin):", len(top_clin))

top_clin.head(20)


Top-200 total: 200
Top-200 clínico (sin admin): 197


,drug_key,reaction_pt,n11,n1dot,ndot1,N,n10,n01,n00,ROR,SE_logROR,ROR_L95,ROR_U95,PRR,is_signal
0,MENTHOL,EXPOSURE TO CHEMICAL POLLUTION,135,182,137,293664,47,2,293480,334876.696842,0.654550,92836.861995,1.207951e+06,108846.346154,True
2,DOLUTEGRAVIR\LAMIVUDINE\TENOFOVIR DISOPROXIL F...,CONGENITAL UMBILICAL HERNIA,14,22,16,293664,8,2,293640,200366.458824,0.765909,44655.037842,8.990412e+05,93431.545455,True
3,BETA GLUCAN,TOTAL LUNG CAPACITY ABNORMAL,18,24,25,293664,6,7,293633,111430.148718,0.584155,35461.738350,3.501430e+05,31461.428571,True
5,CLOFIBRATE,VAGINAL FLATULENCE,15,16,30,293664,1,15,293633,195755.666667,0.892021,34072.996247,1.124653e+06,18353.000000,True
6,ALEFACEPT,NEUROLOGIC NEGLECT SYNDROME,17,20,29,293664,3,12,293632,117453.000000,0.650277,32834.991912,4.201374e+05,20799.783333,True
7,ACLIDINIUM BROMIDE\FORMOTEROL FUMARATE,TOTAL LUNG CAPACITY INCREASED,15,29,17,293664,14,2,293633,125553.634483,0.730401,29998.513697,5.254832e+05,75940.086207,True
8,CALCITRIOL\CALCIUM CARBONATE\ZINC,VAGINAL FLATULENCE,14,15,30,293664,1,16,293633,172027.707071,0.892324,29925.202807,9.889167e+05,17129.525000,True
9,SODIUM STEARATE,VAGINAL FLATULENCE,14,15,30,293664,1,16,293633,172027.707071,0.892324,29925.202807,9.889167e+05,17129.525000,True
10,ALEFACEPT,CAROTID ARTERY THROMBOSIS,17,20,33,293664,3,16,293628,88978.333333,0.635190,25621.198447,3.090076e+05,15599.837500,True
11,BETA GLUCAN,CAPILLARITIS,14,24,19,293664,10,5,293635,73726.662338,0.588239,23275.840426,2.335306e+05,34258.000000,True


In [15]:
# guardamos la versión FULL y la Clínica

out_dir = Path("data_processed")
out_dir.mkdir(exist_ok=True)

top.to_csv(out_dir/"signals_top200_full.csv", index=False)
top_clin.to_csv(out_dir/"signals_top200_clinical.csv", index=False)


### 1. Filtrado por Relevancia Clínica (Severidad)

No todas las reacciones en FAERS son iguales. Categorizar los resultados en:

- Señales Graves: Eventos como "Congenital Umbilical Hernia" (que aparece en tu Top 2) tienen un impacto clínico mayor que "Vaginal Flatulence".

- Señales "Dummy" o de Exposición: Como el caso del Mentol + Exposure to Chemical Pollution. Estas suelen ser asociaciones de uso, no efectos adversos per se. Identificarlas te permite limpiar el ruido.

### 2. Evaluación de la Robustez (Consistencia)

Un ROR alto con muy pocos casos ($n_11$) puede ser un falso positivo. Priorizar:

- El volumen de casos ($n_11$): Una señal con 135 casos (como el Mentol) es mucho más robusta que una con solo 14 casos, aunque el ROR sea similar.

- El Intervalo de Confianza (ROR_L95): Si el límite inferior (L95) es muy cercano a 1, la señal es débil. Si es muy alto (como en tus datos), la asociación es estadísticamente muy "dura".

### 3. Identificación de "Combinaciones Huérfanas"

Buscar medicamentos que aparecen vinculados a reacciones que no están en su prospecto actual.

- Si se encuentra que un fármaco común (ej. Parafina o Sodio) tiene una señal muy alta para algo raro (ej. "Steroid Dependence"), podríamos estar ante un hallazgo de seguridad nuevo.

### 4. Agrupación por Clases Terapéuticas

Obtener una estadística de qué tipos de fármacos dominan el Top:

- ¿Son mayoritariamente suplementos (Beta Glucan, Menthol)?

- ¿Son antirretrovirales (Dolutegravir)?